In [107]:
# import modules
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys

In [108]:
# to keep these words unchangable we use python set
# as our docs are in english we use english stopwords
Stopwords = set(stopwords.words('english'))

In [109]:
def get_unique_word_freq(word_list):
    '''
    takes word list and returns unique-words with their freq in dict
    '''
    unqiue_words=list()
    words_freq={}
    # store unqiue words in list
    for word in word_list:
        if word not in unqiue_words:
            unqiue_words.append(word)
    # get unique words freq from all words list using count
    for unqiue_word in unqiue_words:
        # NOTE: this same method can be used to get freq of a particular word in given document
        words_freq[unqiue_word]=word_list.count(unqiue_word) # used count method of lists
    
    return words_freq

In [110]:
class Node:
    """to init a node with docID and word_freq in that document"""

    def __init__(self,doc_id,word_freq=None):
        self.doc_id=doc_id
        self.word_freq=word_freq
        self.next=None

class LinkedList:
    '''
    init linked list class
    '''
    def __init__(self,head=None):
        self.head=head

In [111]:
test_corpora_dir='test-corpora'
english_corpora_dir='english-corpora'
porter=PorterStemmer()

In [112]:
words_global=list() # to store words from all docs
words_freq_global_dict={} # to store word:freq from all docs
indexed_files={}

for i,file in enumerate(os.listdir(test_corpora_dir)):
    with open(test_corpora_dir+'/'+file,'r') as f:
        corpora_file_str=f.read() # text str

    """remove css code lines"""
    css_regex=re.compile(r'.mw.*}')
    # substitue regex expression by ''
    corpora_file_str=css_regex.sub('',corpora_file_str)

    """remove html tag lines"""
    html_regex=re.compile(r'<.*>')
    # substitue regex expression by ''
    corpora_file_str=html_regex.sub('',corpora_file_str)

    """remove special characters and only keep a-z;A-Z;0-9;space"""
    '''
    to avoid deprecation warning used one more backslash to escape backslash
    so \s -> \\s
    '''
    special_regex=re.compile('[^a-zA-Z0-9\\s]')
    # substitue regex expression by ''
    corpora_file_str=special_regex.sub('',corpora_file_str)

    """remove digits for file"""
    digit_regex=re.compile('\d')
    # substitue regex expression by ''
    corpora_file_str=digit_regex.sub('',corpora_file_str)

    sentence_tokens=sent_tokenize(corpora_file_str)
    # print(len(sentence_tokens))
    word_tokens=word_tokenize(corpora_file_str)
    # print(len(word_tokens))

    """avoid single characters and lower them and remove stopwords"""
    # TODO: what about special cases like UP > up or PIN > pin 
    # TODO: incase of tf-idf stopwords do not matter. 
    # for this should keep them? for cases "like to be or not to be"
    word_tokens=[porter.stem(word.lower()) for word in word_tokens if len(word)>1 and word not in Stopwords]
    # print(len(word_tokens))

    words_freq_global_dict.update(get_unique_word_freq(word_tokens)) # used update method of dict

    indexed_files[i+1]=file

unique_words_global=set(words_freq_global_dict.keys()) # used set to keep words unchangable

<>:20: DeprecationWarning: invalid escape sequence \s
<>:29: DeprecationWarning: invalid escape sequence \d


In [113]:
"""create inverted index linked list with freq count"""
inverted_index_data={}
for word in unique_words_global:
    inverted_index_data[word]=LinkedList()
    inverted_index_data[word].head=Node(1,Node)

"""to use in pivot length normalization we store # of unique terms in a doc"""
unique_words_in_doc={}

"""doc lengths are stored for use in BM25"""
doc_lengths={}

for i,file in enumerate(os.listdir(test_corpora_dir)):
    with open(test_corpora_dir+'/'+file,'r') as f:
        corpora_file_str=f.read() # text str

    """remove css code lines"""
    css_regex=re.compile(r'.mw.*}')
    # substitue regex expression by ''
    corpora_file_str=css_regex.sub('',corpora_file_str)

    """remove html tag lines"""
    html_regex=re.compile(r'<.*>')
    # substitue regex expression by ''
    corpora_file_str=html_regex.sub('',corpora_file_str)

    """remove special characters and only keep a-z;A-Z;0-9;space"""
    '''
    to avoid deprecation warning used one more backslash to escape backslash
    so \s -> \\s
    '''
    special_regex=re.compile('[^a-zA-Z0-9\\s]')
    # substitue regex expression by ''
    corpora_file_str=special_regex.sub('',corpora_file_str)

    """remove digits for file"""
    digit_regex=re.compile('\d')
    # substitue regex expression by ''
    corpora_file_str=digit_regex.sub('',corpora_file_str)

    sentence_tokens=sent_tokenize(corpora_file_str)
    # print(len(sentence_tokens))
    word_tokens=word_tokenize(corpora_file_str)
    # print(len(word_tokens))

    """avoid single characters and lower them and remove stopwords"""
    # TODO: what about special cases like UP > up or PIN > pin 
    # TODO: incase of tf-idf stopwords do not matter. 
    # for this should keep them? for cases "like to be or not to be"
    word_tokens=[porter.stem(word.lower()) for word in word_tokens if len(word)>1 and word not in Stopwords]
    # print(len(word_tokens))

    tmp_word_freq_of_doc=get_unique_word_freq(word_tokens)
    unique_words_in_doc[i+1]=len(tmp_word_freq_of_doc)
    doc_lengths[i+1]=sum(tmp_word_freq_of_doc.values())

    for word in tmp_word_freq_of_doc.keys():
        tmp_LinkedList=inverted_index_data[word].head
        while tmp_LinkedList.next is not None:
            tmp_LinkedList=tmp_LinkedList.next
        tmp_LinkedList.next=Node(i+1,tmp_word_freq_of_doc[word])


In [10]:
import json
with open('inverted-index-data.json','r') as file:
        inverted_index_data=json.load(file)

In [22]:
## a simple check system to check 'not' system
tokenized_query=['not','sunil','prem','james','bond','or','not','ram','shyam']
bool_words=list()
search_words=list()
for i,word in enumerate(tokenized_query):
        if word != "and" and word != "or":
            if word == 'not':
                search_words.append(f'not-{tokenized_query[i+1]}')
            else:
                if tokenized_query[i-1]!='not':
                    search_words.append(word)
                if i!=len(tokenized_query)-1 and tokenized_query[i+1] != 'and' and tokenized_query[i+1]!='or':
                     bool_words.append('and')
        else:   
            bool_words.append(word)
print(search_words)
print(bool_words)

['not-sunil', 'prem', 'james', 'bond', 'not-ram', 'shyam']
['and', 'and', 'and', 'or', 'and']


In [114]:
query_input=input('Query > ')
tokenized_query=[word.lower() for word in word_tokenize(query_input)]


"""seperate logic and search terms"""
bool_words=list()
search_words=list()

for word in tokenized_query:
    if word != "and" and word != "or" and word != "not":
        search_words.append(word)
    else:
        bool_words.append(word)

search_words=[porter.stem(word) for word in search_words]
total_documents=len(indexed_files)

query_word_zero_one=list()
for word in search_words:
    if word in unique_words_global:
        tmp_zero_one=[0]*total_documents
        curr_linkedlist=inverted_index_data[word].head
        while curr_linkedlist.next is not None:
            tmp_zero_one[curr_linkedlist.next.doc_id-1]=1
            curr_linkedlist=curr_linkedlist.next
        query_word_zero_one.append(tmp_zero_one)
    else:
        print(f'word > {word} < is not found in any document')
        # sys.exit() 
"""create a merged boolean(zero-one) list using bitwise operations"""
# try:
for word in bool_words:
    zero_one_list1=query_word_zero_one[0]
    zero_one_list2=query_word_zero_one[1]
    # implement and using '&'
    if word == 'and':
        bitwise_logic=[l1 & l2 for (l1,l2) in zip(zero_one_list1,zero_one_list2)]
        query_word_zero_one.remove(zero_one_list1)
        query_word_zero_one.remove(zero_one_list2)
        query_word_zero_one.insert(0,bitwise_logic)
    # implement or using '|'
    elif word == 'or':
        bitwise_logic=[l1 | l2 for (l1,l2) in zip(zero_one_list1,zero_one_list2)]
        query_word_zero_one.remove(zero_one_list1)
        query_word_zero_one.remove(zero_one_list2)
        query_word_zero_one.insert(0,bitwise_logic)
    # implement not using 'not'
    elif word == 'not':
        bitwise_logic=[int(not l1 == True) for l1 in zero_one_list2]
        query_word_zero_one.remove(zero_one_list2)
        query_word_zero_one.remove(zero_one_list1)
        bitwise_logic=[l1 & l2 for (l1,l2) in zip(zero_one_list1,bitwise_logic)]
    query_word_zero_one.insert(0,bitwise_logic)
# except IndexError:
    # print()
        
query_files_result=list()
try:
    for i,zero_one in enumerate(query_word_zero_one[0]):
        if zero_one==1:
            query_files_result.append(indexed_files[i+1]) # recall indexed_files is a dict
    print(f'{query_input} is present in {len(query_files_result)} files and they are \n {query_files_result}')
except IndexError:
    print(f'No files for query > {query_input}')

javascript and python is present in 6 files and they are 
 ['C00005.txt', 'C00001.txt', 'C00007.txt', 'C00006.txt', 'C00003.txt', 'C00002.txt']


---
---

### tf-idf implementation

In [115]:
for i,word in enumerate(unique_words_global):
    if i<5:
        l=inverted_index_data[word].head
        while l.next is not None:
            print(l.next.doc_id,end=' ')
            l=l.next
        print('======')
        # print()

2 ======
1 3 4 5 6 7 8 9 ======
3 ======
6 ======
1 2 3 4 5 6 7 8 9 ======


In [122]:
"""implement tf-idf using simple/basic definition"""

def get_tf_idf(term,document_id):
    '''
    takes term and doc-id
    and return tf-idf for term and document
    If doc_id is not in doc_ids then it is zero
    '''
    doc_ids=list()
    # word_freqs=list()

    # get term appearning doc-ids using inverted index linked list
    tmp=inverted_index_data[term].head
    while tmp.next is not None:
        doc_ids.append(tmp.next.doc_id)
        # since no need to extract word freq for all documents in which the term appears
        if doc_ids[-1]==document_id:
            term_freq_in_doc=tmp.next.word_freq
        tmp=tmp.next
    if document_id not in doc_ids:
        return [0,0]
    else:
        idf=np.log(total_documents/len(doc_ids))
        # tf=term_freq_in_doc # since tf is just simple count of how many times term appears in particular document

        return [term_freq_in_doc*idf,term_freq_in_doc]

In [136]:
# query_input=input('Query > ')
processed_query=[porter.stem(word.lower()) for word in word_tokenize(query_input)]
# print(processed_query)
doc_scores={}
for doc in indexed_files.keys():
    tmp_tf_idfs=[]
    for word in processed_query:
        foo,faa=get_tf_idf(word,doc)
        tmp_tf_idfs.append(foo)

    doc_scores[doc]=sum(tmp_tf_idfs)/unique_words_in_doc[doc]


In [23]:
sorted_doc_score_dict=dict(sorted(doc_scores.items(), key=lambda item: item[1],reverse=True))

NameError: name 'doc_scores' is not defined

In [138]:
print(f'For "{query_input}" top five docs are:')
for i,doc_id in enumerate(sorted_doc_score_dict.keys()):
    if i<5:
        print(f'{i+1}. {indexed_files[doc_id]}')

For "javascript and python" top five docs are:
1. C00005.txt
2. C00001.txt
3. C00002.txt
4. C00007.txt
5. C00009.txt


---
---

### bm25 implementation

In [140]:
b=0.75
k1=1.5
# avg document length
avdl=sum(doc_lengths.values())/total_documents

# query_input=input('Query > ')
processed_query=[porter.stem(word.lower()) for word in word_tokenize(query_input)]
# print(processed_query)
doc_scores={}
for doc in indexed_files.keys():
    tmp_tf_idfs=[]
    tmp_tfs=[]
    for word in processed_query:
        foo,faa=get_tf_idf(word,doc) # foo: tf-idf, faa: tf
        tmp_tf_idfs.append(foo)
        tmp_tfs.append(faa)

    # for RSV formula please see markdowns
    doc_scores[doc]=sum([((k1+1)*i)/((k1*(1-b+(b*(doc_lengths[doc]/avdl))))+j) for i,j in zip(tmp_tf_idfs,tmp_tfs)]) 


In [141]:
sorted_doc_score_dict=dict(sorted(doc_scores.items(), key=lambda item: item[1],reverse=True))
print(f'For "{query_input}" top five docs are:')
for i,doc_id in enumerate(sorted_doc_score_dict.keys()):
    if i<5:
        print(f'{i+1}. {indexed_files[doc_id]}')

For "javascript and python" top five docs are:
1. C00002.txt
2. C00005.txt
3. C00007.txt
4. C00001.txt
5. C00003.txt


- TODO: pre-process query also as we pre-process corpus for tf-idf and BM25 case
- TODO: how to rank for boolean model?
- TODO: how to find relevance from scores